In [3]:
from datetime import datetime, timedelta

def days_30_months_inclusive(start_date, end_date):
    # Convert dates to datetime objects
    start = datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.strptime(end_date, "%Y-%m-%d")
    
    # Calculate year and month differences
    years = end.year - start.year
    months = end.month - start.month
    days = end.day - start.day
    
    # Adjust days difference if it's negative (borrow from months)
    if days < 0:
        days += 30
        months -= 1
    
    # Adjust months difference if it's negative (borrow from years)
    if months < 0:
        months += 12
        years -= 1
    
    # Total days, assuming 360 days per year and 30 days per month
    # Adding 1 to make the calculation inclusive of both start and end dates
    total_days = years * 360 + months * 30 + days + 1
    
    # Return both the total days and the breakdown
    return total_days, {"Years": years, "Months": months, "Days": days + 1}

def subtract_30_day_months(start_date, days):
    # Convert the start date to a datetime object
    start = datetime.strptime(start_date, "%Y-%m-%d")
    
    # Calculate total months and remaining days from the given days
    total_months = days // 30
    remaining_days = days % 30
    
    # Calculate the new year and month
    new_year = start.year - total_months // 12
    new_month = start.month - total_months % 12
    
    # Adjust the year if the month calculation results in months less than 1
    if new_month < 1:
        new_year -= 1
        new_month += 12
    
    # Create a new date with adjusted month and subtract the remaining days
    new_date = datetime(new_year, new_month, start.day) - timedelta(days=remaining_days)
    
    return new_date.strftime("%Y-%m-%d"), {
        "Years": total_months // 12,
        "Months": total_months % 12,
        "Days": remaining_days
    }

def calculate_total_days_and_result_from_periods(date_ranges, reference_start_date):
    # Calculate the total days from all given date ranges and get breakdowns
    total_days = 0
    breakdowns = []
    for start_date, end_date in date_ranges:
        days, breakdown = days_30_months_inclusive(start_date, end_date)
        breakdowns.append({
            "Period": f"{start_date} to {end_date}",
            "Days": days,
            "Breakdown": breakdown
        })
        total_days += days

    # Subtract those days from the reference start date and get the breakdown
    result_date, result_breakdown = subtract_30_day_months(reference_start_date, total_days)

    # Return the final date, total days, and detailed breakdowns
    return result_date, total_days, breakdowns, result_breakdown

# Define the date ranges for testing
date_ranges = [
    ("2008-08-04", "2013-08-03"),
    ("2013-08-04", "2015-09-23")
]

# Reference start date
reference_start_date = "2021-02-01"

# Get the result, total days, and detailed breakdowns
final_result_date, total_days_inclusive, detailed_breakdowns, final_result_breakdown = calculate_total_days_and_result_from_periods(
    date_ranges, reference_start_date
)

# Output the detailed results
print(f"Resulting date using the 30-day month calculation is: {final_result_date}")
print(f"Total Days (Inclusive): {total_days_inclusive}")
print("Detailed Breakdown by Periods:")
for breakdown in detailed_breakdowns:
    print(f"{breakdown['Period']}: {breakdown['Days']} days, Breakdown: {breakdown['Breakdown']}")
print(f"Final Breakdown of Days Subtracted: {final_result_breakdown}")


Resulting date using the 30-day month calculation is: 2013-12-12
Total Days (Inclusive): 2570
Detailed Breakdown by Periods:
2008-08-04 to 2013-08-03: 1800 days, Breakdown: {'Years': 4, 'Months': 11, 'Days': 30}
2013-08-04 to 2015-09-23: 770 days, Breakdown: {'Years': 2, 'Months': 1, 'Days': 20}
Final Breakdown of Days Subtracted: {'Years': 7, 'Months': 1, 'Days': 20}


In [7]:
from datetime import datetime, timedelta
import ipywidgets as widgets
from IPython.display import display

# Function to calculate the total days using 30-day month assumptions
def days_30_months_inclusive(start_date, end_date):
    start = datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.strptime(end_date, "%Y-%m-%d")
    
    years = end.year - start.year
    months = end.month - start.month
    days = end.day - start.day
    
    # Adjust days if negative by borrowing 30 days from months
    if days < 0:
        days += 30
        months -= 1
    
    # Adjust months if negative by borrowing 12 months from years
    if months < 0:
        months += 12
        years -= 1
    
    # Calculate the total days, assuming 360 days per year and 30 days per month
    total_days = years * 360 + months * 30 + days + 1  # +1 for inclusivity
    
    return total_days, {"Years": years, "Months": months, "Days": days + 1}

# Function to subtract days using the 30-day month model
def subtract_30_day_months(start_date, days):
    start = datetime.strptime(start_date, "%Y-%m-%d")
    total_months = days // 30
    remaining_days = days % 30
    
    # Calculate the new year and month
    new_year = start.year - total_months // 12
    new_month = start.month - total_months % 12
    
    # Adjust if new_month goes below 1
    if new_month < 1:
        new_year -= 1
        new_month += 12
    
    # Create a new date with adjusted year and month, and subtract remaining days
    new_date = datetime(new_year, new_month, start.day) - timedelta(days=remaining_days)
    
    return new_date.strftime("%Y-%m-%d"), {
        "Years": total_months // 12,
        "Months": total_months % 12,
        "Days": remaining_days
    }

# Function to calculate total days and result date from periods
def calculate_total_days_and_result_from_periods(date_ranges, reference_start_date):
    total_days = 0
    breakdowns = []
    for start_date, end_date in date_ranges:
        days, breakdown = days_30_months_inclusive(start_date, end_date)
        breakdowns.append({
            "Period": f"{start_date} to {end_date}",
            "Days": days,
            "Breakdown": breakdown
        })
        total_days += days

    # Subtract those days from the reference start date and get the breakdown
    result_date, result_breakdown = subtract_30_day_months(reference_start_date, total_days)
    return result_date, total_days, breakdowns, result_breakdown

# Create a widget for the reference start date
reference_date_widget = widgets.Text(
    value='2021-02-01',
    description='Reference Start Date (YYYY-MM-DD):',
    disabled=False
)

# Create widgets for up to 5 periods with start and end date fields
periods = []
for i in range(1, 6):
    period_label = widgets.HTML(value=f"<b>Period {i}</b>")
    start_date_widget = widgets.Text(value='', description='Start Date (YYYY-MM-DD):', disabled=False)
    end_date_widget = widgets.Text(value='', description='End Date (YYYY-MM-DD):', disabled=False)
    periods.append((period_label, start_date_widget, end_date_widget))

# Button to calculate results
button = widgets.Button(description="Calculate")

# Function to handle button click
def on_button_click(b):
    reference_date = reference_date_widget.value
    date_ranges = []
    
    # Gather all date ranges from the widgets
    for i, (label, start_date_widget, end_date_widget) in enumerate(periods):
        start_date = start_date_widget.value
        end_date = end_date_widget.value
        
        # Skip empty fields
        if not start_date or not end_date:
            continue
        
        date_ranges.append((start_date, end_date))
    
    # Calculate the total days and resulting date
    try:
        final_result_date, total_days_inclusive, detailed_breakdowns, final_result_breakdown = calculate_total_days_and_result_from_periods(
            date_ranges, reference_date
        )
        print(f"Resulting date using 30-day month calculation is: {final_result_date}")
        print(f"Total Days (Inclusive): {total_days_inclusive}")
        print("Detailed Breakdown by Periods:")
        for breakdown in detailed_breakdowns:
            print(f"{breakdown['Period']}: {breakdown['Days']} days, Breakdown: {breakdown['Breakdown']}")
        print(f"Final Breakdown of Days Subtracted: {final_result_breakdown}")
    except ValueError:
        print("Invalid date format. Please use YYYY-MM-DD format for all inputs.")

# Link the button to the calculation function
button.on_click(on_button_click)

# Display the widgets
display(reference_date_widget)
for widgets_set in periods:
    display(*widgets_set)
display(button)


Text(value='2021-02-01', description='Reference Start Date (YYYY-MM-DD):')

HTML(value='<b>Period 1</b>')

Text(value='', description='Start Date (YYYY-MM-DD):')

Text(value='', description='End Date (YYYY-MM-DD):')

HTML(value='<b>Period 2</b>')

...

HTML(value='<b>Period 5</b>')

Text(value='', description='Start Date (YYYY-MM-DD):')

Text(value='', description='End Date (YYYY-MM-DD):')

Button(description='Calculate', style=ButtonStyle())

Resulting date using 30-day month calculation is: 2013-12-12
Total Days (Inclusive): 2570
Detailed Breakdown by Periods:
2008-08-04 to 2013-08-03: 1800 days, Breakdown: {'Years': 4, 'Months': 11, 'Days': 30}
2013-08-04 to 2015-09-23: 770 days, Breakdown: {'Years': 2, 'Months': 1, 'Days': 20}
Final Breakdown of Days Subtracted: {'Years': 7, 'Months': 1, 'Days': 20}
Resulting date using 30-day month calculation is: 2013-12-11
Total Days (Inclusive): 2570
Detailed Breakdown by Periods:
2008-08-04 to 2013-08-03: 1800 days, Breakdown: {'Years': 4, 'Months': 11, 'Days': 30}
2013-08-04 to 2015-09-23: 770 days, Breakdown: {'Years': 2, 'Months': 1, 'Days': 20}
Final Breakdown of Days Subtracted: {'Years': 7, 'Months': 1, 'Days': 20}
